<a href="https://colab.research.google.com/github/variableVG/CV_project/blob/master/06_PyTorch_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Data and images


In [22]:
import pandas as pd
import numpy as np
import torchvision.transforms as T
import matplotlib.pyplot as plt
from PIL import Image
import torch
import os

In [23]:
# Use Google Collabs GPU
def get_device():
  if torch.cuda.is_available():
    return torch.device('cuda')
  else:
    return torch.device('cpu')

device = get_device()

def to_device(data, device=device):
  if isinstance(data, (list, tuple)):
    return [to_device(x, device) for x in data]
  return data.to(device, non_blocking=True)



In [24]:
path_to_pictures = '/content/pic/'
path_to_dataset = '/content/drive/MyDrive/Colab Notebooks/CVNLP/dataset_faces_small.csv'

In [25]:
#Get .tar file with pytorch files ffrom google drive
! pip install gdown
! mkdir -p /content/pic/
! gdown -O /content/pytorch_files.tar "16MCrByNUvuVrn_PpAEjrD5_sKGFwvTBE"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/uc?id=16MCrByNUvuVrn_PpAEjrD5_sKGFwvTBE
To: /content/pytorch_files.tar
100% 248M/248M [00:01<00:00, 215MB/s]


In [26]:
import tarfile
try: 
  tar = tarfile.open('/content/pytorch_files.tar')
  tar.extractall(path='/content/pic')
  tar.close()
except:
  print("Pictures could not be extracted")

In [27]:
! mv /content/pic/content/drive/MyDrive/Colab\ Notebooks/CVNLP/normalized_tensor_images/* /content/pic/
! rmdir /content/pic/content

rmdir: failed to remove '/content/pic/content': Directory not empty


In [28]:
df = pd.read_csv(path_to_dataset)

In [29]:
df.head()

,filename,age,gender,ethnicity
0,36_1_1_20170116153744623.jpg,36,1,1
1,37_0_1_20170116002833730.jpg,37,0,1
2,37_0_1_20170117143125092.jpg,37,0,1
3,37_1_1_20170116001322849.jpg,37,1,1
4,37_1_1_20170116232758932.jpg,37,1,1


In [30]:
df.shape

(1251, 4)

# Create Dataset and Load Pictures with Data Loader

Because my RAM was constantly crashing and taking a long time to upload photos, I created a class that uses torch.load() and loads directly the images as torch files which seems more efficient

In [31]:
# SOurce: https://pytorch.org/docs/stable/data.html
# We want a class that implements the image dataset to be able to request pictures
# by need and not load all pictures at once (RAM crashes)

class TensorDataset(torch.utils.data.Dataset): 
  def __init__(self, df):
    super().__init__()
    self.df = df

  def __len__(self): 
    return len(self.df)

  def __getitem__(self, idx): # by __getitem__ return a tupel of (picture, label)
    basename, extension = os.path.splitext(self.df['filename'][idx])
    path = path_to_pictures + basename + '.pth'
    torch_picture = torch.load(path)
    label = self.df['age'][idx]
    return (torch_picture, label)

In [32]:
# Create tensor dataset 
tensor_dataset = TensorDataset(df)
print(len(tensor_dataset))

1251


In [33]:
img, label = tensor_dataset.__getitem__(3)
print(img.shape, label)
# Element at index 3 is a pytorch image with age 37. 

torch.Size([3, 128, 128]) 37


In [34]:
# split dataset with PyTorch
test_size = int(len(tensor_dataset) * 0.2)
train_size = len(tensor_dataset) - test_size
print(f"Dataset has length {len(tensor_dataset)}, train set has length {train_size} and test set has length {test_size}.")
print(f"Sum of test and train sets is {train_size + test_size}.")

Dataset has length 1251, train set has length 1001 and test set has length 250.
Sum of test and train sets is 1251.


In [35]:
from torch.utils.data import random_split
# Random numbers will be generated always on the same way, so we can compare models. 
random_seed = 42
torch.manual_seed(random_seed)

# Split dataset
train_ds, test_ds = random_split(tensor_dataset, [train_size, test_size])
print(len(train_ds))
print(len(test_ds))

1001
250


In [36]:
# Load Data with DataLoader
batch_size=8

from torch.utils.data import DataLoader
train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_ds, batch_size)

# Create Model

In [37]:
# Define initial number of kernels. 
num_pixels_x = 128
num_pixels_y = 128
num_kernels = 16

In [38]:
from torch import nn

class Model(nn.Module): 
  def __init__(self): 
    super().__init__()
    self.network = nn.Sequential(
        nn.Conv2d(3, num_kernels, kernel_size=3, padding=1),
        nn.ReLU(), 
        nn.Conv2d(num_kernels, num_kernels*2, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2), 
        #MaxPool reduce the size of the image by half (if it was 32x32, becomes 16x16)

        nn.Conv2d(num_kernels*2, num_kernels*3, kernel_size=3, padding=1, stride=1),
        nn.ReLU(), 
        nn.Conv2d(num_kernels*3, num_kernels*3, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2), 

        nn.Conv2d(num_kernels*3, num_kernels*4, kernel_size=3, stride=1, padding=1),
        nn.ReLU(), 
        nn.Conv2d(num_kernels*4, num_kernels*4, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        # MaxPool2d reduces the image of a picture by half
        nn.MaxPool2d(2,2), # Output:8x4096

        # Flatten() reduces the 3 dimensions in one dimension. 
        nn.Flatten(), 
        nn.Linear(num_pixels_x*num_pixels_y*num_kernels // 16, num_kernels*4), 
        nn.ReLU(),
        nn.Linear(num_kernels*4, num_kernels*2),
        nn.ReLU(),
        nn.Linear(num_kernels*2, 1), 
        #nn.ReLU(), # To make sure there is at the end a number > 0
    )
  def forward(self, x):
    return self.network(x)

# Train the model

In [39]:
def fit(model, train_loader, num_epochs, optimizer, loss_function):
  #log = {}
  # Tell the model we are in training mode. 
  # model.train()

  for epoch in range(num_epochs): # for each number of epochs. 
    model.train()
    sumloss = []
    for x, y in train_loader: # for each batch in the train loader: 
      x, y = to_device([x, y])
      prediction = model(x) # The prediction is 3x1 but we need a tensor of 3.
      y = y[:, None]
      #prediction = prediction.squeeze()
      loss = loss_function(prediction, y)
      sumloss.append(loss.detach().item())
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    #log[epoch] = sum(losses) / len(losses)
    print(f"Epoch [{epoch + 1 }/{num_epochs}] loss: {np.mean(sumloss):.3f}.")


In [40]:
# Initialize the model
model = to_device(Model())

number_param = 0
for param in model.parameters():
  number_param = number_param + np.prod(param.shape)

print(number_param)

1155137


In [44]:
# Create Optimizer
optimizer = torch.optim.Adam(model.parameters())
# Other optimizer could be the stochastic gradient descent
# torch.optim.SGD(model.parameters(), lr=1e-5)

# define loss function
loss_function = nn.L1Loss()

# Define num of epochs: 
num_epochs = 50

# Train the model using the function above: 
fit(model, train_loader, num_epochs, optimizer, loss_function)

Epoch [1/50] loss: 16.753676694536967.
Epoch [2/50] loss: 16.730683674888006.
Epoch [3/50] loss: 16.581802171374125.
Epoch [4/50] loss: 16.53394649520753.
Epoch [5/50] loss: 16.375661744011772.
Epoch [6/50] loss: 16.070507723187642.
Epoch [7/50] loss: 16.265874669665383.
Epoch [8/50] loss: 16.11109665461949.
Epoch [9/50] loss: 15.472288985101004.
Epoch [10/50] loss: 15.62151811614869.
Epoch [11/50] loss: 16.167609714326403.
Epoch [12/50] loss: 15.268537385123116.
Epoch [13/50] loss: 15.049605952368843.
Epoch [14/50] loss: 15.2012459020766.
Epoch [15/50] loss: 14.58672607134259.
Epoch [16/50] loss: 14.53099909282866.
Epoch [17/50] loss: 14.261441484330193.
Epoch [18/50] loss: 13.85767676338317.
Epoch [19/50] loss: 13.370558413248213.
Epoch [20/50] loss: 13.446772787306044.
Epoch [21/50] loss: 13.267713501339866.
Epoch [22/50] loss: 13.478022068265885.
Epoch [23/50] loss: 14.162138647503323.
Epoch [24/50] loss: 12.511676201744685.
Epoch [25/50] loss: 12.01255072487725.
Epoch [26/50] loss

In [45]:
def evaluate(loader, name="Training"):
  # Get predictions
  absolute_error, total = 0, 0 

  # This tells PyTorch that the model is in evalution mode. 
  model.eval() 
  for idx, (x, y) in enumerate(loader):
    x, y = to_device((x,y))
    prediction = model(x).squeeze()
    #max_prob, prediction = torch.max(output, 1)
    total += y.size(0)
    absolute_error += ((prediction - y).abs()).sum().item()

  mean_absolute_error = absolute_error/ total 
  print(f"{name} => Mean Absolute error is {mean_absolute_error:.2f} years")

evaluate(train_loader)
evaluate(test_loader, "Testing")

Training => Mean Absolute error is 4.94 years
Testing => Mean Absolute error is 18.89 years


In [43]:
def predict():

SyntaxError: ignored